In [46]:
from curses import meta
import os
import time
import json
import streamlit as st
import pandas as pd
import openai
from langchain.document_loaders import Docx2txtLoader

OPENAI_API_KEY = st.secrets.get("OPENAI_API_KEY")
TRANSCRIPT_PATH = "data/transcripts"
ASSISTANT_ID = "asst_XfSqQlAPl7opg5fsMCjE9lfL"
transcript_files = os.listdir(TRANSCRIPT_PATH)



### Create a data file with all transcript filenames and raw text.

In [47]:
transcripts = []  # List to store file and transcript information

for file in transcript_files:
    if file.endswith('.docx'):  # Process only .docx files
        doc = Docx2txtLoader(os.path.join(TRANSCRIPT_PATH, file))
        transcript = doc.load()[0].page_content
        transcripts.append({"file": file, "transcript": transcript})

# Create DataFrame from the collected data
transcripts_df = pd.DataFrame(transcripts, columns=["file", "transcript"])
transcripts_df.head(5)

,file,transcript
0,Sean Taylor Catapult-X Educator Interview - De...,"This transcript was exported on Dec 18, 2023 -..."
1,Sarah Dorey Webster Catapult-X Educator Interv...,"This transcript was exported on Dec 18, 2023 -..."
2,Neil T. McGovern Catapult-X Educator Interview...,"This transcript was exported on Dec 18, 2023 -..."
3,Amanda Fuller CB Amazon loyalist_Millennial.docx,"This transcript was exported on Dec 18, 2023 -..."
4,Kelda Bailess Catapult-X Educator Interview - ...,"This transcript was exported on Dec 18, 2023 -..."


### Create a table with transcripts and key survey data

In [51]:
import re
survey_data = pd.read_csv("data/survey_data.csv")

for index, row in transcripts_df.iterrows():
    # Extract last name from file name
    file_name = row['file']
    transcript = row['transcript']
    
    # Separate keywords with whitespace characters, hyphens, or underscores
    keywords = re.findall(r"[\w'-]+", file_name)
    
    # filter keywords to first two name-sized words
    keywords = [keyword for keyword in keywords if len(keyword) >= 3]
    first_name = keywords[0]
    last_name = keywords[1]
    
    # find survey data for this person by filtering each name
    filtered_data = survey_data[survey_data['RecipientFirstName'].str.contains(first_name, case=False) & survey_data['RecipientLastName'].str.contains(last_name, case=False)]
    
    if filtered_data.empty:
        print(f"Could not find survey data for {first_name} {last_name}")
        continue

    # add survey data to transcript dataframe: Age Group, Email, Role
    transcripts_df.loc[index, 'ResponseId'] = filtered_data['ResponseId'].values[0]
    transcripts_df.loc[index, 'FirstName'] = filtered_data['RecipientFirstName'].values[0]
    transcripts_df.loc[index, 'LastName'] = filtered_data['RecipientLastName'].values[0]
    transcripts_df.loc[index, 'Email'] = filtered_data['RecipientEmail'].values[0]
    transcripts_df.loc[index, 'AgeGroup'] = filtered_data['Q15'].values[0]
    transcripts_df.loc[index, 'InstitutionName'] = filtered_data['InstitutionName'].values[0]
    transcripts_df.loc[index, 'District'] = filtered_data['ParentName'].values[0]
    transcripts_df.loc[index, 'City'] = filtered_data['MailingCity'].values[0]
    transcripts_df.loc[index, 'State'] = filtered_data['MailingState'].values[0]
    transcripts_df.loc[index, 'Role'] = filtered_data['Q2'].values[0]
    transcripts_df.loc[index, 'Subjects'] = filtered_data['Q3'].values[0]
    transcripts_df.loc[index, 'Courses'] = filtered_data['Q4'].values[0]
    transcripts_df.loc[index, 'TopOfMind'] = filtered_data['Q5'].values[0]
    transcripts_df.loc[index, 'Carolina Familiarity'] = filtered_data['Q6_1'].values[0]
    transcripts_df.loc[index, 'Fisher Familiarity'] = filtered_data['Q6_2'].values[0]
    transcripts_df.loc[index, 'Flinn Scientific Familiarity'] = filtered_data['Q6_3'].values[0]
    transcripts_df.loc[index, 'PLTW Familiarity'] = filtered_data['Q6_4'].values[0]
    transcripts_df.loc[index, 'Sargent Welch Familiarity'] = filtered_data['Q6_5'].values[0]
    transcripts_df.loc[index, 'Thomas Scientific Familiarity'] = filtered_data['Q6_6'].values[0]
    transcripts_df.loc[index, 'Wards/VWR Familiarity'] = filtered_data['Q6_7'].values[0]
    transcripts_df.loc[index, 'BioRad Familiarity'] = filtered_data['Q6_8'].values[0]
    transcripts_df.loc[index, 'BioCorp Familiarity'] = filtered_data['Q6_9'].values[0]
    transcripts_df.loc[index, 'Amazon Familiarity'] = filtered_data['Q6_10'].values[0]
    transcripts_df.loc[index, 'Nasco Familiarity'] = filtered_data['Q6_11'].values[0]
    transcripts_df.loc[index, 'Frey/School Specialty Familiarity'] = filtered_data['Q6_12'].values[0]
    transcripts_df.loc[index, 'Primary Vendor'] = filtered_data['Q7'].values[0]
    transcripts_df.loc[index, 'Top Vendor Qualities'] = filtered_data['Q8'].values[0]
    transcripts_df.loc[index, 'Years in Eduacation'] = filtered_data['Q14'].values[0]
transcripts_df

# save the dataframe to a csv file
transcripts_df.to_csv("data/transcripts.csv", index=False)
    

### Create a table for relevant interview segments, with timestamps, transcript segment, themes, and survey data.

In [ ]:
transcript = transcripts_df['transcript'].iloc[0]

functions = {
    "name": "add_transcript_with_annotations",
    "description": "Adds a complete transcript record along with its annotations to the database.",
    "parameters": {
        "type": "object",
        "properties": {
            "interviewee_name": {
                "type": "string",
                "description": "The name of the interviewee."
            },
            "interviewee_role": {
                "type": "string",
                "description": "The role of the interviewee (e.g., Teacher, Administrator)."
            },
            "interviewee_institution": {
                "type": "string",
                "description": "The institution or organization the interviewee is affiliated with."
            },
            "date_of_interview": {
                "type": "string",
                "description": "The date when the interview was conducted."
            },
            "other_metadata": {
                "type": "object",
                "description": "Any other relevant metadata about the transcript."
            },
            "annotations": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "text_segment": {
                            "type": "string",
                            "description": "The specific segment of text being annotated."
                        },
                        "themes": {
                            "type": "array",
                            "items": {
                                "type": "string",
                                "description": "The theme(s) identified in this segment, such as: Brand Perception, Product Quality, Customer Service, Purchasing Experience, Digital Resources, Environmental Sustainability, Educational Policies, Customer Experience, Buying Habits, Purchasing Patterns, Vendor Comparison, Budget and Timing, Generational Insights"
                            }
                        },
                        "brands": {
                            "type": "array",
                            "items": {
                                "type": "string",
                                "description": "The brand(s) identified in this segment, such as: Carolina, Flinn, or Amazon."
                            }
                        },
                        "time_code": {
                            "type": "string",
                            "description": "The time code in the transcript for this text segment."
                        }
                    }
                },
                "description": "An array of annotation objects associated with the transcript."
            }
        },
        "required": [
            "interviewee_name"
        ]
    }
}

instructions = """As Transcript Pro, analyze educator interview transcripts for market research, focusing solely on interviewee statements and disregarding comments by Daylene or Kimberly. Key areas include:

1. **Brand Perception:** Views on brands like Carolina, Flinn Scientific, Amazon.
2. **Product Quality:** Discussions about product durability, effectiveness, quality.
3. **Customer Service:** Experiences with customer service.
4. **Purchasing Experience:** Ease or difficulty in purchasing.
5. **Digital Resources:** Use of digital/virtual teaching tools.
6. **Environmental Sustainability:** Eco-friendly practices in education.
7. **Educational Policies:** Policy influence on purchases.
8. **Customer Experience:** Brand experiences, positive or negative.
9. **Buying Habits:** Timing and methods of buying.
10. **Purchasing Patterns:** What is bought from various vendors.
11. **Vendor Comparison:** Comparisons between Carolina Biological and others.
12. **Budget and Timing:** Budget and purchase timing considerations.
13. **Generational Insights:** Generational differences in buying.

Focus on processing and summarizing interviewee statements accurately and objectively, maintaining consistency in coding.
"""

